In [1]:
import pandas as pd
import numpy as np

# read in the final dataframe

In [2]:
df = pd.read_csv('final_df.csv')

/Users/siyuan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df = df.drop(['Unnamed: 0','type.1'],axis=1)

# Only use the selected 30 features from Feature Selection

## code for feature selection is in ISE540_projectcode_PART4_featureselection.ipynb

In [5]:
top_30 = ['author_num_helpful_votes',
 'service',
 'cleanliness',
 'overall',
 'value',
 'sleep_quality',
 'rooms',
 'average_service_rating',
 'average_overall_rating',
 'average_value_rating',
 'average_sleep_quality_rating',
 'average_rooms_rating',
 'min_service_rating',
 'min_cleanliness_rating',
 'min_overall_rating',
 'min_value_rating',
 'min_sleep_quality_rating',
 'min_rooms_rating',
 'max_service_rating',
 'max_overall_rating',
 'total_number_helpful_reviews_hotel_got',
 'number_of_hotels_in_region',
 'number_of_hotels_in_class_1',
 'number_of_hotels_in_class_1.5',
 'number_of_hotels_in_class_2',
 'number_of_hotels_in_class_3',
 'number_of_hotels_in_class_3.5',
 'number_of_hotels_in_class_4',
 'number_of_hotels_in_class_4.5',
 'number_of_hotels_in_class_5']

In [161]:
y_train.value_counts()/sum(y_train)

1.0    0.301869
2.0    0.133560
3.0    0.105916
4.0    0.023091
5.0    0.003982
6.0    0.000165
Name: votes, dtype: float64

# Modeling  - Baseline Model

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
df['adjectives_text'] = df['adjectives_text'].fillna('')
df['title_adj_word'] = df['title_adj_word'].fillna('')

In [7]:
df['title_review_adj'] = df['title_adj_word'] + df['adjectives_text']

In [8]:
df['title_review'] = df['title'] + df['text']

In [9]:
df.loc[df['num_helpful_votes'] ==0, 'votes'] = 1
df.loc[df['num_helpful_votes'] ==1, 'votes'] = 2
df.loc[(df['num_helpful_votes'] >=2) & (df['num_helpful_votes']<= 5), 'votes'] = 3
df.loc[(df['num_helpful_votes'] >=6) & (df['num_helpful_votes']<= 13), 'votes'] = 4
df.loc[(df['num_helpful_votes'] >=14) & (df['num_helpful_votes']<= 36), 'votes'] = 5
df.loc[df['num_helpful_votes'] >=37, 'votes'] = 6

In [93]:
X_train,X_test,y_train,y_test = train_test_split(df.drop('votes',axis = 1),df['votes'],
                                                test_size = 0.3)

In [1]:
from sklearn.linear_model import LogisticRegression

In [ ]:
x_train_baseline = X_train.copy()
x_test_baseline = X_test.copy()

In [ ]:
model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=10000)
model.fit(X_train, y_train)
model.score(X_train, y_train),model5.score(X_test, y_test)

In [ ]:
x_train_model2 = x_train_baseline
x_test_model2 = x_test_baseline
from scipy.sparse import hstack
for i in top_30:
    x_train_model2 = hstack((x_train_model2,np.array(X_train[i])[:,None]))
    x_test_model2 = hstack((x_test_model2,np.array(X_test[i])[:,None]))

In [ ]:
model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=10000)
model.fit(x_train_model2, y_train)
model.score(x_train_model2, y_train),model5.score(x_test_model2, y_test)

# Modeling - Random Guessing 

In [ ]:
p_list = []
for i in list(y_train.value_counts()/sum(y_train)):
    p_list.append(i/sum(list(y_train.value_counts()/sum(y_train))))

In [ ]:
p_list

In [ ]:
random_guessing_train = []
for i in range(len(X_train)):
    random_guessing_train.append(np.random.choice(np.arange(1, 7), p=p_list))

In [ ]:
random_guessing_test = []
for i in range(len(X_test)):
    random_guessing_test.append(np.random.choice(np.arange(1, 7), p=p_list))

In [ ]:
for i in range(1,7):
    print(random_guessing_train.count(i)/len(random_guessing_train))

In [ ]:
for i in range(1,7):
    print(random_guessing_test.count(i)/len(random_guessing_test))

In [ ]:
[accuracy_score(list(y_train),random_guessing_train),
accuracy_score(list(y_test),random_guessing_test)]
# 0.37330546420244876,0.37329268347601313 

In [ ]:
[recall_score(y_test,random_guessing_test,average='weighted'),
 precision_score(y_test,random_guessing_test,average='weighted')]
# [0.37329268347601313, 0.37343828427776293]

# Model - Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(max_depth=20)
model.fit(X_train, y_train)
model.score(X_train, y_train),model5.score(X_test, y_test)

In [ ]:
model = RandomForestClassifier(max_depth=20)
model.fit(x_train_model2, y_train)
model.score(x_train_model2, y_train),model5.score(x_test_model2, y_test)

## Final Model - XGBoost 

### XGB - more models are saved in later sections

In [97]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

x_trainn = scaler.fit_transform(x_train)
x_testt = scaler.transform(x_test)

In [98]:
from xgboost import XGBClassifier

In [101]:
model5 = XGBClassifier(learning_rate =0.01, max_depth=15,n_estimators=1500)
model5.fit(x_trainn, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1500, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [102]:
model5.score(x_trainn, y_train),model5.score(x_testt, y_test)

(0.9335309648432072, 0.603916076358085)

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
pca = TruncatedSVD(n_components=50)
x_train_model2_new = pca.fit_transform(x_train_baseline)
x_test_model2_new = pca.transform(x_test_baseline)

In [ ]:
x_train_model2_new = pd.DataFrame(x_train_model2_new)
x_test_model2_new = pd.DataFrame(x_test_model2_new)

In [ ]:
for i in top_30:
    x_train_model2_new[i] = list(X_train[i])
    x_test_model2_new[i] = list(X_test[i])

## Tune the model 

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [ ]:
param_comb = 5

model5 = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, 
                            scoring='roc_auc', n_jobs=4, verbose=3, random_state=1001)
model5.fit(x_trainn, y_train)

In [104]:
y_pre=model5.predict(x_testt)

In [110]:
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [114]:
accuracy_score(y_test,y_pre)

0.603916076358085

In [108]:
recall_score(y_test,y_pre,average='weighted')
precision_score(y_test,y_pre,average='weighted')

0.603916076358085

In [112]:
precision_score(y_test,y_pre,average='weighted')

0.5872947737021991

In [117]:
temppp = pd.DataFrame(zip(list(y_pre),list(y_test)),columns= ['pre','true'])

In [125]:
accuracy_score(temppp.pre,temppp.true)

0.6917260728004719

In [128]:
recall_score(temppp.pre,temppp.true,average='weighted')

0.6917260728004719

In [130]:
precision_score(temppp.pre,temppp.true,average='weighted')

0.765192484443968

# Model - Decision Tree classifiers

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.multiclass import OneVsRestClassifier

In [27]:
pca = TruncatedSVD(n_components=100)
x_train_baseline = pca.fit_transform(x_train_baseline)
x_test_baseline = pca.transform(x_test_baseline)

In [ ]:
regressor = OneVsRestClassifier(DecisionTreeClassifier(criterion='entropy',max_depth=10))
regressor = regressor.fit(x_train_baseline,y_train)

In [40]:
regressor.score(x_train_baseline,y_train)

0.5334410270662528

In [21]:
regressor = OneVsRestClassifier(DecisionTreeClassifier(criterion='entropy',max_depth=10))
regressor = regressor.fit(x_train_baseline,y_train)

In [22]:
regressor.score(x_train_baseline,y_train)

0.5323081198758627

In [23]:
regressor.score(x_test_baseline,y_test)

0.5287918976488231

In [24]:
x_train_model2 = x_train_baseline
x_test_model2 = x_test_baseline

In [25]:
x_train_model2 = x_train_baseline
x_test_model2 = x_test_baseline
from scipy.sparse import hstack
for i in top_30:
    x_train_model2 = hstack((x_train_model2,np.array(X_train[i])[:,None]))
    x_test_model2 = hstack((x_test_model2,np.array(X_test[i])[:,None]))

In [33]:
regressor = OneVsRestClassifier(DecisionTreeClassifier(criterion='entropy',max_depth=10))
regressor = regressor.fit(x_train_model2,y_train)

In [34]:
regressor.score(x_train_model2,y_train)

0.6004890607871293

In [35]:
regressor.score(x_test_model2,y_test)

0.5922795293142813

# Model - GradientBoosting Classifier

In [39]:
from sklearn.ensemble import GradientBoostingClassifier

In [40]:
tfidf = TfidfVectorizer(lowercase=True,stop_words='english')
x_train_baseline = tfidf.fit_transform(X_train['title_review_adj'])
x_test_baseline = tfidf.transform(X_test['title_review_adj'])

In [45]:
Classifier = OneVsRestClassifier(GradientBoostingClassifier(max_depth =10))
Classifier = Classifier.fit(x_train_baseline,y_train)

In [46]:
Classifier.score(x_train_baseline,y_train)

0.533659116525391

In [47]:
Classifier.score(x_test_baseline,y_test)

0.5288594472640155

In [49]:
x_train_model2 = x_train_baseline
x_test_model2 = x_test_baseline

In [50]:
from scipy.sparse import hstack
for i in top_30:
    x_train_model2 = hstack((x_train_model2,np.array(X_train[i])[:,None]))
    x_test_model2 = hstack((x_test_model2,np.array(X_test[i])[:,None]))

In [51]:
Classifier1 = OneVsRestClassifier(GradientBoostingClassifier(max_depth =10))
Classifier1 = Classifier1.fit(x_train_model2,y_train)

In [53]:
Classifier1.score(x_train_model2,y_train)

0.6265285562091806

In [54]:
Classifier1.score(x_test_model2,y_test)

0.5981563458360165

# Model - XGBoost

In [60]:
from sklearn.model_selection import GridSearchCV

In [55]:
from xgboost import XGBClassifier

In [64]:
model1 = XGBClassifier(learning_rate =0.1, max_depth=10)
model1.fit(x_train_model2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [65]:
model2 = XGBClassifier(learning_rate =0.1, max_depth=15)
model2.fit(x_train_model2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [66]:
model1.score(x_train_model2, y_train),model2.score(x_train_model2, y_train)

(0.622031667361465, 0.6917025645776399)

In [67]:
model1.score(x_test_model2, y_test),model2.score(x_test_model2, y_test)

(0.599876609369582, 0.5981428359129781)

In [68]:
model3 = XGBClassifier(learning_rate =0.1, max_depth=15,n_estimators=1000)
model3.fit(x_train_model2, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [98]:
model3.score(x_train_model2, y_train),model3.score(x_test_model2, y_test)

(0.7481877344944184, 0.595058070152527)

In [106]:
tfidf = TfidfVectorizer(lowercase=True,stop_words='english')
x_train_baseline = tfidf.fit_transform(X_train['title_review_adj'])
x_test_baseline = tfidf.transform(X_test['title_review_adj'])

In [107]:
pca = TruncatedSVD(n_components=50)
x_train_baseline = pca.fit_transform(x_train_baseline)
x_test_baseline = pca.transform(x_test_baseline)

In [108]:
x_train_baseline_new = pd.DataFrame(x_train_baseline)
x_test_baseline_new = pd.DataFrame(x_test_baseline)

In [109]:
for i in top_30:
    x_train_baseline_new[i] = list(X_train[i])
    x_test_baseline_new[i] = list(X_test[i])

In [110]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

x_train_baseline_new = scaler.fit_transform(x_train_baseline_new)
x_test_baseline_new = scaler.transform(x_test_baseline_new)

In [111]:
x_train_baseline_new = pd.DataFrame(x_train_baseline_new)
x_test_baseline_new = pd.DataFrame(x_test_baseline_new)

In [112]:
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest,chi2

In [113]:
sel_chi2 = SelectKBest(chi2, k=20)    # select 30 features
x_train_baseline_new_2 = sel_chi2.fit_transform(x_train_baseline_new, y_train)
print(sel_chi2.get_support())

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True  True False  True  True False  True  True  True False
 False False  True False  True  True False  True False False  True  True
  True  True False  True  True  True  True  True]


In [114]:
x_test_baseline_new = sel_chi2.transform(x_test_baseline_new)

In [115]:
x_train_baseline_new = pd.DataFrame(x_train_baseline_new_2)
x_test_baseline_new = pd.DataFrame(x_test_baseline_new)

In [99]:
model4 = XGBClassifier(learning_rate =0.1, max_depth=15,n_estimators=1000)
model4.fit(x_train_baseline_new, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [100]:
model4.score(x_train_baseline_new, y_train),model4.score(x_test_baseline_new, y_test)

(0.9948276128275202, 0.5898162200136)

In [117]:
model5 = XGBClassifier(learning_rate =0.01, max_depth=15,n_estimators=1500)
model5.fit(x_train_baseline_new, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1500, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [118]:
model5.score(x_train_baseline_new, y_train),model5.score(x_test_baseline_new, y_test)

(0.7671113375638828, 0.589798206782882)

# Model tried to improve Linear regression - not used in this project

## Linear regression baseline model - use only adjectives in `review`

In [13]:
tfidf = TfidfVectorizer(lowercase=True,stop_words='english')
x_train_baseline = tfidf.fit_transform(X_train['adjectives_text'])
x_test_baseline = tfidf.transform(X_test['adjectives_text'])

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [15]:
pca = TruncatedSVD(n_components=100)
x_train_baseline = pca.fit_transform(x_train_baseline)
x_test_baseline = pca.transform(x_test_baseline)

In [16]:
model = LinearRegression()
model = model.fit(x_train_baseline,y_train)

In [17]:
[mean_squared_error(y_train,model.predict(x_train_baseline)),
mean_squared_error(y_test,model.predict(x_test_baseline))]

[8.277395195413414, 10.8405137539474]

## Linear regresison model - use adj in `review` + `title`

In [20]:
X_train['title_review_adj'] = X_train['title_adj_word'] + X_train['adjectives_text']
X_test['title_review_adj'] = X_test['title_adj_word'] + X_test['adjectives_text']

In [21]:
tfidf = TfidfVectorizer(lowercase=True,stop_words='english')
x_train_model1 = tfidf.fit_transform(X_train['title_review_adj'])
x_test_model1 = tfidf.transform(X_test['title_review_adj'])

In [22]:
pca = TruncatedSVD(n_components=100)
x_train_model1 = pca.fit_transform(x_train_model1)
x_test_model1 = pca.transform(x_test_model1)

In [23]:
model = LinearRegression()
model = model.fit(x_train_model1,y_train)

In [24]:
[mean_squared_error(y_train,model.predict(x_train_model1)),
mean_squared_error(y_test,model.predict(x_test_model1))]

[8.276740443000426, 10.839120477160291]

## Linear regression model - use `review` + `title` adj + numerical columns

In [29]:
x_train_model2 = pd.DataFrame(x_train_model1)
x_test_model2 = pd.DataFrame(x_test_model1)

In [30]:
for i in top_30:
    x_train_model2[i] = list(X_train[i])
    x_test_model2[i] = list(X_test[i])

In [136]:
#from scipy.sparse import hstack
#for i in top_30:
#    x_train_model2 = hstack((x_train_model2,np.array(X_train[i])[:,None]))
#    x_test_model2 = hstack((x_test_model2,np.array(X_test[i])[:,None]))

In [32]:
model = LinearRegression()
model = model.fit(x_train_model2,y_train)

In [33]:
[mean_squared_error(y_train,model.predict(x_train_model2)),
mean_squared_error(y_test,model.predict(x_test_model2))]

[8.169609935108497, 10.730003954939047]

## Linear regression model - use `review` + `title` adj + numerical columns + count adv features

In [35]:
x_train_model3 = x_train_model2
x_test_model3 = x_test_model2

In [36]:
for i in ['title_adv_count','title_exclamation_count', 'title_questionmark_count',
       'review_exclamation_count', 'review_questionmark_count']:
    x_train_model3[i] = list(X_train[i])
    x_test_model3[i] = list(X_test[i])

In [144]:
#for i in ['title_adv_count','title_exclamation_count', 'title_questionmark_count',
#       'review_exclamation_count', 'review_questionmark_count']:
#    x_train_model3 = hstack((x_train_model3,np.array(X_train[i])[:,None]))
#    x_test_model3 = hstack((x_test_model3,np.array(X_test[i])[:,None]))

In [37]:
model = LinearRegression()
model = model.fit(x_train_model3,y_train)

In [38]:
[mean_squared_error(y_train,model.predict(x_train_model3)),
mean_squared_error(y_test,model.predict(x_test_model3))]

[8.13519043934702, 10.691889181789378]

## Decision Tree regressor baseline model - use only adjectives in `review`

In [42]:
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, train_test_split
from sklearn.tree import DecisionTreeRegressor

In [43]:
tfidf = TfidfVectorizer(lowercase=True,stop_words='english')
x_train_baseline = tfidf.fit_transform(X_train['adjectives_text'])
x_test_baseline = tfidf.transform(X_test['adjectives_text'])

In [44]:
pca = TruncatedSVD(n_components=100)
x_train_baseline = pca.fit_transform(x_train_baseline)
x_test_baseline = pca.transform(x_test_baseline)

In [50]:
dtm = DecisionTreeRegressor(max_depth = 8,max_leaf_nodes= 100,
                            min_samples_leaf = 20,min_samples_split = 10)
dtm = dtm.fit(x_train_baseline,y_train)

In [51]:
[mean_squared_error(y_train,dtm.predict(x_train_baseline)),
mean_squared_error(y_test,dtm.predict(x_test_baseline))]

[8.184743918155984, 10.892728443751531]

## Linear regresison model - use adj in `review` + `title`

In [60]:
dtm = DecisionTreeRegressor(max_depth = 8,max_leaf_nodes= 100,
                            min_samples_leaf = 20,min_samples_split = 10)
dtm = dtm.fit(x_train_model1,y_train)

In [61]:
[mean_squared_error(y_train,dtm.predict(x_train_model1)),
mean_squared_error(y_test,dtm.predict(x_test_model1))]

[8.187151710870488, 10.951390385016152]

In [63]:
dtm = DecisionTreeRegressor(max_depth = 8,max_leaf_nodes= 100,
                            min_samples_leaf = 20,min_samples_split = 10)
dtm = dtm.fit(x_train_model2,y_train)

In [64]:
[mean_squared_error(y_train,dtm.predict(x_train_model2)),
mean_squared_error(y_test,dtm.predict(x_test_model2))]

[7.918191176738325, 10.644244020179377]

In [118]:
dtm = DecisionTreeRegressor(max_depth = 8,max_leaf_nodes= 100,
                            min_samples_leaf = 20,min_samples_split = 10)
dtm = dtm.fit(x_train_model3,y_train)

In [119]:
[mean_squared_error(y_train,dtm.predict(x_train_model3)),
mean_squared_error(y_test,dtm.predict(x_test_model3))]

[7.918191176738325, 10.644244020179377]